# Network Structure

There are a variety of ways to store networks, with significant differences in storage requirements and information access rates. In general, the formats may be considered as link-oriented or nodeoriented. Link-oriented formats facilitate access to node information given a link index; nodeoriented formats facilitate access to link parameters given node indices. Four simple storage formats are presented: (1) Node-link Incidence Matrix, (2) Adjacency Matrix, (3) Ladder Representation, and (4) Forward Star.

## Dependencies

In [94]:
import os
import numpy as np
import pandas as pd
from sys import getsizeof

## Sample Networks
<img src="./Data/TransportationNetworks\Seervada Park Management\network_cost.png">


In [95]:
links_df=pd.read_csv("./Data/Seervada Park Management\links.csv")
links_df

,FID,Id,FNode,TNode,Dist,Cost,FName,TName
0,0,0,0,1,70.710678,2,R,A
1,1,1,0,2,150.000000,5,R,B
2,2,2,0,3,111.803399,4,R,C
3,3,3,1,2,111.803399,2,A,B
4,5,4,1,4,206.155281,7,A,D
5,6,5,2,4,100.000000,4,B,D
6,7,6,2,5,111.803399,3,B,E
7,4,7,3,2,70.710678,1,C,B
8,11,8,3,5,150.000000,4,C,E
9,9,9,4,6,104.403065,5,D,S


## 1. Node-Arc Incidence Matrix

The Node-Arc Incidence Matrix comprises N rows that correspond to network nodes and M columns that represent network links. For a given link (i,j), the matrix is coded as: 

Cell [node n, link(i,j)]

    -1 if n = i (node n is the origin of link (i,j)) 
    
     1 if n = j (node n is the destination of link (i,j)) 

    0 otherwise Matrix Properties:
    
    

Requires N by M storage locations (about 4N2 for traffic networks).
2M entries of N are nonzero

Each column has exactly one +1 and one -1.

The node-arc incident matrix can be easily implemented to standard linear programming that is well known as the standard minimum cost network flow problem. We will revisit this problem in a notebook in the optimization chapter.

In [96]:
def generate_node_arc_incidence_matrix(df):
    nodes=np.concatenate((df["FNode"].as_matrix()[:], df["TNode"].as_matrix()[:]))
    nodes = np.unique(nodes)
    NAIM = np.zeros((len(nodes), len(links_df)))
    links = df[["FNode", "TNode","Id"]].as_matrix()
    for fnode, tnode, idx in links:
        NAIM[fnode][idx]=1
        NAIM[tnode][idx]=-1
    return NAIM
generate_node_arc_incidence_matrix(links_df)

array([[ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0., -1.,  0.,  1.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,  1., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -1.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0., -1.]])

## 2. Node-Node Adjacency Matrix
The Adjacency Matrix is an N1 by N2 matrix in which each of N1 rows and N2 columns corresponds to the A-node and B-node of a given link (i,j) in the network, coded as follows: 
    
    Cell [i,j] =

    1 if link (i,j) is in the network 
    0 if link (i,j) is not in the network Matrix Properties: N2 elements, M of which are non-zero
    
The (row, column) location of the cell, not the value contained, indicates that the link exists; thus, the matrix can be used to store costs and/orcapacities [e.g., Cij, Lij, or Uij in cell(i,j)]


In [97]:
NNAM_nodes = [[0,1,1,1,0,0,0],
              [0,0,1,0,1,0,0],
              [0,0,0,0,1,1,0],
              [0,0,1,0,0,1,0],
              [0,0,0,0,0,0,1],
              [0,0,0,0,0,0,1],
              [0,0,0,0,0,0,0]
             ]
NNAM_nodes = np.array(NNAM_nodes)
print (getsizeof(NNAM_nodes))
print (NNAM_nodes.nbytes)

308
196


The Node-Node Adjacency Matrix has a intereteting property that enable to check network connectivity.
For example, $A\cdot A$ shows how each node can be reached within two connections. This implies that the matrix of $A^k$ indicates how a node is visited in k number of link trips. The number in the matrix shows the number of ways to reach the node with k connections.  

In [98]:
np.dot(NNAM_nodes, NNAM_nodes)

array([[0, 0, 2, 0, 2, 2, 0],
       [0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 2],
       [0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

The matrix above shows $A\cdot A$ case. In here, the value of matrix (1,2) is 2, meaning that there are two different ways to reach the node from R and A in two link trips.  

## 3. Ladder Representation
A network can be represented as a listing of links, typically following the ladder format: index A-Node B-Node [ tij Uij etc.] where: index = pointer for link number A-node= starting node of the link B-node= ending node of the link. Any link characteristics can be included, including link cost tij, and link capacity, Uij. The network topography is stored in two M-vectors (A-node and B-node). For traffic applications, this may be approximated as 8N. 

In [99]:
Ladder_LINK=links_df
print (Ladder_LINK)
print (getsizeof(Ladder_LINK))

    FID  Id  FNode  TNode        Dist  Cost FName TName
0     0   0      0      1   70.710678     2     R     A
1     1   1      0      2  150.000000     5     R     B
2     2   2      0      3  111.803399     4     R     C
3     3   3      1      2  111.803399     2     A     B
4     5   4      1      4  206.155281     7     A     D
5     6   5      2      4  100.000000     4     B     D
6     7   6      2      5  111.803399     3     B     E
7     4   7      3      2   70.710678     1     C     B
8    11   8      3      5  150.000000     4     C     E
9     9   9      4      6  104.403065     5     D     S
10    8  10      5      4   50.000000     1     E     D
11   10  11      5      6  128.062485     7     E     S
2168


## 4. Foward Star Representation
A N-vector of link origin nodes serves as pointers to a M-vector of link terminal nodes, providing an efficient network storage requirement of N+M (about 5N for traffic network applications). Each link origin node is assigned a value in the first vector which serves as a pointer to the location in the second vector corresponding to the beginning of a group of links, each with the same origin node and ordered by their terminal node. Define A-node(i) as the list of link origin nodes and B-node(j) as the list of link terminal nodes, ordered in groups by origin node. For node i, find the pointers for A-node(i) and A-node(i+1) then find the terminal node locations in list B-node(⋅) at A-node(i) to Anode(i+1)-1.

In [100]:
links_df.sort_values(by=["FNode"])

,FID,Id,FNode,TNode,Dist,Cost,FName,TName
0,0,0,0,1,70.710678,2,R,A
1,1,1,0,2,150.000000,5,R,B
2,2,2,0,3,111.803399,4,R,C
3,3,3,1,2,111.803399,2,A,B
4,5,4,1,4,206.155281,7,A,D
5,6,5,2,4,100.000000,4,B,D
6,7,6,2,5,111.803399,3,B,E
7,4,7,3,2,70.710678,1,C,B
8,11,8,3,5,150.000000,4,C,E
9,9,9,4,6,104.403065,5,D,S


In [116]:
def generate_foward_star_matrix(df):
    df = links_df
    nodes=np.concatenate((df["FNode"].as_matrix()[:], df["TNode"].as_matrix()[:]))
    nodes = np.unique(nodes)
    f_star = np.full((len(nodes), 2), np.inf)
    links = df[["Id", "FNode", "TNode"]].as_matrix()
    for i in links:
        ID, fnode, tnode = i[0], i[1],i[2]
        if f_star[fnode][1]==np.inf:
            f_star[fnode][0]=fnode
            f_star[fnode][1]=ID
    f_star[-1][0]=nodes[-1]
    f_star[-1][1]=len(links)
    return f_star
generate_foward_star_matrix(links_df)

array([[  0.,   0.],
       [  1.,   3.],
       [  2.,   5.],
       [  3.,   7.],
       [  4.,   9.],
       [  5.,  10.],
       [  6.,  12.]])

## 5. Python Class-Dictionary Representation


In [118]:
class Graph: #Data structure
    def __init__(self):
        self.nodes = {}
        self.edges = {}
        self.costs = {}

In [119]:
def add_node(self, idx, coordinates=[], **kwargs):
        self.nodes[idx]={}
        if len (coordinates) >0:
            self.nodes[idx]["loc"]= coordinates
        if kwargs is not None:
            for key, value in kwargs.items():
                self.nodes[idx][key] = value
Graph.add_node = add_node

In [120]:
g = Graph()
g.add_node("A", coordinates=[1,3],index="1")
g.nodes

{'A': {'index': '1', 'loc': [1, 3]}}

In [121]:
def add_edge(self, from_node, to_node, cost=1, bidirection=False, **kwargs):
    if not from_node in self.edges:
        self.edges[from_node] = {}
    if bidirection==True:
        if not to_node in self.edges:
            self.edges[to_node] = {}

    self.edges[from_node][to_node]={"cost":cost}
    if kwargs is not None:
        for key, value in kwargs.items():
            self.edges[from_node][to_node][key] = value
    self.costs[(from_node, to_node)] = cost
    if from_node not in self.nodes:
        self.add_node(from_node)
    if to_node not in self.nodes:
        self.add_node(to_node)
        
Graph.add_edge = add_edge

In [122]:
g.add_edge("A","B",cost=1, distance=2)
print (g.edges)
print (g.nodes)

{'A': {'B': {'distance': 2, 'cost': 1}}}
{'A': {'loc': [1, 3], 'index': '1'}, 'B': {}}


In [123]:
linkset = links_df[["FName","TName","Cost"]].as_matrix()
gh_t = Graph()
for l in linkset:
    snode, enode, cost = l[0],l[1], l[2]
    gh_t.add_edge(snode, enode, cost)

gh_t.edges

{'A': {'B': {'cost': 2}, 'D': {'cost': 7}},
 'B': {'D': {'cost': 4}, 'E': {'cost': 3}},
 'C': {'B': {'cost': 1}, 'E': {'cost': 4}},
 'D': {'S': {'cost': 5}},
 'E': {'D': {'cost': 1}, 'S': {'cost': 7}},
 'R': {'A': {'cost': 2}, 'B': {'cost': 5}, 'C': {'cost': 4}}}